In [1]:
import yaml
from data.eua_dataset import get_dataset
from util.torch_utils import seed_torch

with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)
dir_name = config['train']['dir_name']
total_sec = config['data']['total_sec']
user_num_per_sec = 10
user_stay_miu = 50
user_stay_sigma = 5
max_cov = 1.5
miu = 35
sigma = 10
data_size = {
    'test': 1000
}

seed_torch(42)
data_set = get_dataset(0.5, 1, miu, sigma, total_sec, user_num_per_sec, user_stay_miu, user_stay_sigma, data_size, max_cov, 'cpu', dir_name)

读取服务器数据以及用户位置和mask数据成功
正在加载 test 数据集


In [2]:
servers = data_set['test'].servers_tensor

In [3]:
print(servers.shape)

torch.Size([65, 6])


In [4]:
import torch
import torch.nn as nn
from dgl.nn.pytorch import GraphConv

class GNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GNN, self).__init__()
        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, out_dim)

    def forward(self, g, features, edge_weight=None):
        # 第一层图卷积
        x = self.conv1(g, features, edge_weight=edge_weight)
        x = torch.relu(x)
        # 第二层图卷积
        x = self.conv2(g, x, edge_weight=edge_weight)
        return x

In [5]:
import dgl


def create_graph_from_server_list(server_list, device):
    # 服务器节点数量
    num_servers = len(server_list)

    # 创建服务器图，有两个特征，分别是position，即server_list中的前两个维度，和capacity，即后4个维度
    g = dgl.graph(([], []), num_nodes=num_servers, device=device)
    g.ndata['position'] = server_list[:, :2]
    g.ndata['capacity'] = server_list[:, 2:]
    
    # 添加边
    # 使用矩阵计算服务器之间的距离
    positions_servers = g.ndata['position']
    server_positions_expand = positions_servers.unsqueeze(0)
    server_distances_matrix = torch.cdist(server_positions_expand, server_positions_expand, p=2)

    # 添加服务器之间的边
    src_servers, dst_servers = torch.meshgrid(torch.arange(num_servers, device=device),
                                              torch.arange(num_servers, device=device), indexing='ij')
    g.add_edges(src_servers.flatten(), dst_servers.flatten())

    # 赋予服务器之间的边特征，即距离
    g.edata['distance'] = server_distances_matrix.flatten()

    # 删除自己指向自己的边，防止距离为0，输入的参数是eid
    # 所以要先计算自己的eid，计算方法是对角线元素的下标
    e_ids = torch.arange(num_servers, device=device) * num_servers + torch.arange(num_servers, device=device)
    g.remove_edges(e_ids)

    # 继续计算所有距离特征（包括服务器之间的和服务器到用户的）的倒数（+1是为了避免除0）作为权重
    g.edata['weight'] = 1 / (g.edata['distance'] + 1).float()

    return g

In [6]:
graph = create_graph_from_server_list(servers, 'cpu')

In [7]:
print(graph)

Graph(num_nodes=65, num_edges=4160,
      ndata_schemes={'position': Scheme(shape=(2,), dtype=torch.float32), 'capacity': Scheme(shape=(4,), dtype=torch.float32)}
      edata_schemes={'distance': Scheme(shape=(), dtype=torch.float32), 'weight': Scheme(shape=(), dtype=torch.float32)})


In [8]:
# 创建模型
in_dim = 2 + 4
hidden_dim = 16
out_dim = 1
model = GNN(in_dim, hidden_dim, out_dim)

In [9]:
# 编码
features = torch.cat([graph.ndata['position'], graph.ndata['capacity']], dim=1)

# 前向传播
outputs = model(graph, features, edge_weight=graph.edata['weight'])
print(outputs)

tensor([[0.2606],
        [0.4382],
        [0.3108],
        [0.3979],
        [0.3350],
        [0.4635],
        [0.4428],
        [0.3004],
        [0.3939],
        [0.4195],
        [0.4342],
        [0.4666],
        [0.4725],
        [0.3940],
        [0.4354],
        [0.2983],
        [0.5064],
        [0.4040],
        [0.4553],
        [0.3748],
        [0.4368],
        [0.2634],
        [0.3463],
        [0.3863],
        [0.4006],
        [0.4936],
        [0.4671],
        [0.3674],
        [0.3267],
        [0.3342],
        [0.3453],
        [0.4842],
        [0.4436],
        [0.3962],
        [0.4879],
        [0.3849],
        [0.3105],
        [0.4052],
        [0.3615],
        [0.3768],
        [0.4930],
        [0.5039],
        [0.2759],
        [0.5041],
        [0.2440],
        [0.2986],
        [0.4805],
        [0.4231],
        [0.4888],
        [0.4670],
        [0.4965],
        [0.3554],
        [0.4879],
        [0.3512],
        [0.3968],
        [0

In [10]:
print(graph.ndata['position'].shape)

torch.Size([65, 2])


In [11]:
graph.ndata['position'] = graph.ndata['position'].unsqueeze(1).repeat(1, 10, 1)
print(graph.ndata['position'].shape)
graph.ndata['capacity'] = graph.ndata['capacity'].unsqueeze(1).repeat(1, 10, 1)
print(graph.ndata['capacity'].shape)

torch.Size([65, 10, 2])
torch.Size([65, 10, 4])


In [12]:
features2 = torch.cat([graph.ndata['position'], graph.ndata['capacity']], dim=2)
print(features2.shape)

torch.Size([65, 10, 6])


In [13]:
outputs2 = model(graph, features2, edge_weight=graph.edata['weight'])

In [14]:
print(outputs2)

tensor([[[0.2606],
         [0.2606],
         [0.2606],
         [0.2606],
         [0.2606],
         [0.2606],
         [0.2606],
         [0.2606],
         [0.2606],
         [0.2606]],

        [[0.4382],
         [0.4382],
         [0.4382],
         [0.4382],
         [0.4382],
         [0.4382],
         [0.4382],
         [0.4382],
         [0.4382],
         [0.4382]],

        [[0.3108],
         [0.3108],
         [0.3108],
         [0.3108],
         [0.3108],
         [0.3108],
         [0.3108],
         [0.3108],
         [0.3108],
         [0.3108]],

        [[0.3979],
         [0.3979],
         [0.3979],
         [0.3979],
         [0.3979],
         [0.3979],
         [0.3979],
         [0.3979],
         [0.3979],
         [0.3979]],

        [[0.3350],
         [0.3350],
         [0.3350],
         [0.3350],
         [0.3350],
         [0.3350],
         [0.3350],
         [0.3350],
         [0.3350],
         [0.3350]],

        [[0.4635],
         [0.4635],
  

In [15]:
torch.all(outputs.unsqueeze(1).repeat(1, 10, 1) == outputs2)

tensor(True)

In [16]:
import numpy as np

ten_server_capacity_list = np.random.normal(miu, sigma, size=(10, 65, 4))

In [17]:
ten_server_capacity_list = torch.tensor(ten_server_capacity_list, dtype=torch.float32)

In [18]:
# 创建服务器图，有两个特征，分别是position，即server_list中的前两个维度，和capacity，即后4个维度
position = servers[:, :2]
# 创建10个图
graphs = []
for i in range(10):
    servers_i = torch.cat([position, ten_server_capacity_list[i]], dim=1)
    g = create_graph_from_server_list(servers_i, 'cpu')
    graphs.append(g)

In [19]:
# 对10个图进行前向传播，保存10个输出
outputs10 = []
for i in range(10):
    features = torch.cat([graphs[i].ndata['position'], graphs[i].ndata['capacity']], dim=1)
    outputs10.append(model(graphs[i], features, edge_weight=graphs[i].edata['weight']))

In [20]:
outputs10 = torch.stack(outputs10, dim=1)

In [21]:
# 再次测试，把10个图的特征拼接起来，然后进行前向传播
position10 = servers[:, :2].unsqueeze(1).repeat(1, 10, 1)
print(position10.shape)

torch.Size([65, 10, 2])


In [22]:
print(ten_server_capacity_list.shape)

torch.Size([10, 65, 4])


In [23]:
features10 = torch.cat([position10, ten_server_capacity_list.transpose(0, 1)], dim=2)

In [24]:
print(features10.shape)

torch.Size([65, 10, 6])


In [25]:
outputs10_2 = model(graph, features10, edge_weight=graph.edata['weight'])

In [26]:
print(torch.all(outputs10 == outputs10_2))

tensor(True)
